In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

## Import files "written"
written_train = np.load("written_train.npy", allow_pickle=True)
written_train = written_train

# # Import files "spoken"
spoken_train = np.load('spoken_train.npy', allow_pickle=True)
spoken_train = spoken_train
#spoken_train = []

# for array in spoken_array:
#     x = array.flatten()
#     spoken_train.append(x)

# # Import file "match"
match_train = np.load('match_train.npy', allow_pickle=True)
match_train = match_train

In [2]:
lengths_spoken = np.empty(len(spoken_train))
for index, value in enumerate(spoken_train):
    lengths_spoken[index] = value.shape[0]

sorted_lengths = np.sort(lengths_spoken)
longest_length = sorted_lengths[-1]
print(longest_length)

padden_spoken = []

for i in spoken_train:
    if i.shape[0] < longest_length:
        pad_size = longest_length - i.shape[0]
        length = i.shape[1]
        pad_array = np.zeros((13,), dtype=np.int)
        x = np.tile(pad_array, (int(pad_size),1))
        i = np.concatenate((i, x), axis=0)
        padden_spoken.append(i)
    else:
        padden_spoken.append(i)

93.0


In [3]:
nieuw = np.array(padden_spoken)
nieuw_2 = []
for i in range(0,len(nieuw)):
    temp_list = np.array(nieuw[i].flatten())
    nieuw_2.append(temp_list)

nieuw_3 = np.array(nieuw_2)
nieuw_4 = np.concatenate((nieuw_3, written_train), axis=1)

In [4]:
target = []
indexes_true = []
indexes_false = []
                       
for i, j in enumerate(match_train):
    if j == True:
        target.append(1)
        indexes_true.append(i)
    elif j == False:
        target.append(-1)
        indexes_false.append(i)
        
false_included = indexes_false[:6000]
included_rows = indexes_true + false_included

In [ ]:
import pandas as pd
df_a=pd.DataFrame(data=nieuw_4[:,:],    # values
             index=range(0,nieuw_4.shape[0]),    # 1st column as index
             columns=range(0,nieuw_4.shape[1]))

In [ ]:
X = df_a.iloc[included_rows, 0:1993].values
y = np.array(target)[included_rows]

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size = 0.2)

In [ ]:
import keras
from keras.models import Sequential
from keras.layers import Dense

In [ ]:
classifier = Sequential()

# Adding the input layer and the first hidden layer
#classifier.add(Flatten())
classifier.add(Dense(output_dim = 6, init = 'uniform', activation = 'relu', input_dim = 6))
# Adding the second hidden layer
classifier.add(Dense(output_dim = 4, init = 'uniform', activation = 'relu'))
classifier.add(Dense(output_dim = 6, init = 'uniform', activation = 'relu'))
classifier.add(Dense(output_dim = 4, init = 'uniform', activation = 'relu'))
# Adding the output layer
classifier.add(Dense(output_dim = 1, init = 'uniform', activation = 'sigmoid'))

In [ ]:
classifier.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

In [ ]:
classifier.fit(X_train, y_train, batch_size = 50, nb_epoch = 100)

In [ ]:
loss, accuracy = classifier.evaluate(X_val, y_val)
print("Accuracy", accuracy)

In [ ]:
## TN, FP
## FN, TP
y_pred = classifier.predict(X_val)
y_predict = (y_pred > 0.5)

from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_val, y_predict)
print(cm)